# 1: Base Data Inspection

Very concise summary of findings, see Old Notebooks/ for previous work

Feel free to minimise from Base Data Inspection, I got lost down rabbit holes before I realised this was a machine learning subject not feature engineering. 

That said I found there was ~100,000 duplicate rows, 0 duplicated columns, 0 constant columns. 

Chose not to remove the duplicate rows as, I was to *consider that the amount of data for each species in the database available is an indication of its abundance or rarity*

Making it a Hierarchal Multiclassification, edge detection problem, given the scarcity of specific wood types, the target to inditify would be less likely to have duplicates then others, resulting in more complicated classification in this scenario.

In [ ]:
from pathlib import Path
def squish(INS: Path, OUT: Path, CLASS_HEADERS: list[str]):      
    files = [
"Data\BGLBP.parquet",
"Data\CSLBP.parquet",
"Data\CSSILTP.parquet",
"Data\OLBP.parquet",
"Data\SCSLBP.parquet",
"Data\SILTP.parquet",
"Data\Tchebyshev.parquet",
]
    dfs = [pl.read_parquet(f) for f in files]
    master = dfs[0]
    for df in dfs[1:]:
        feat_cols = [c for c in df.columns if c.lower() not in CLASS_HEADERS]
        master = master.hstack(df.select(feat_cols))
    master.write_parquet(OUT)

In [ ]:
import polars as pl
BASE = Path("Data")
RAW = Path("Data/Raw")
CLASS_HEADERS = ["family", "genus", "species"]
for file in RAW.glob("*.csv"):
    name = file.stem.split("_")[1]
    print(name)
    df = pl.read_csv(file, has_header = False)
    ncols = len(df.columns) 
    new_headers = CLASS_HEADERS + [f"{name}_{i}" for i in range(1, ncols - len(CLASS_HEADERS) + 1)]
    df = df.rename(dict(zip(df.columns, new_headers)))
    df.write_parquet(BASE / f"{name}.parquet")
    print(f"Rows x Cols: {len(df)} x {ncols}")

In [ ]:
import polars as pl
MASTER = BASE / "master.parquet"
squish(BASE, MASTER, CLASS_HEADERS)
data = pl.read_parquet(MASTER)
data.describe()

### Prelim Analysis

In [ ]:
data = data.to_pandas

In [ ]:
FIGURESIZE = (10,6)
FONTSIZE = 18
newdata = data.copy
newdata["index"] = range(1, len(newdata) + 1)
np_data = newdata.loc[:,["family", "genus", "species", "index"]].to_numpy()
family, genus, species, index = np_data[:,0], np_data[:,1], np_data[:,2], np_data[:, 3]
nind = len(newdata["index"])

plt.figure(figsize=FIGURESIZE)
plt.plot(index, family, "r", label="Family")
plt.plot(index, genus, "g", label="Genus")
plt.plot(index, species, "b", label="Species")
plt.fill_between(index, family, 0, color="r", alpha=0.3)
plt.fill_between(index, genus, family, color="g", alpha=0.3)
plt.fill_between(index, species, genus, color="b", alpha=0.3)
plt.ylabel("Instance as Table Index", fontsize=FONTSIZE)
plt.xlabel("Index", fontsize=FONTSIZE)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=FIGURESIZE)
plt.plot(index, family, "r", label="Family")
plt.plot(index, genus, "g", label="Genus")
plt.plot(index, species, "b", label="Species")
plt.xlim(0, nind)
plt.ylim(-10, 70)
plt.fill_between(index, family, 0, color="r", alpha=0.3)
plt.fill_between(index, genus, family, color="g", alpha=0.3)
plt.fill_between(index, species, genus, color="b", alpha=0.3)
plt.ylabel("Instance as Table Index", fontsize=FONTSIZE)
plt.xlabel("Index", fontsize=FONTSIZE)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
for file in files:
    df = pd.read_parquet(file)
    print(file)
    matrix = df.corr()
    classes = ["family", "genus", "species"]
    features = [c for c in df.columns if c not in classes]
    df.hist(column = features, xlabelsize = 5, ylabelsize = 5, figsize = (10,6), bins = 21)
    plt.figure(figsize=(40,40))
    sns.heatmap(matrix, annot = True, cmap = "coolwarm", fmt = ".2f", linewidths = 0.5)
    plt.title(f"Correlation Matrix: {file}")

In [ ]:
df = data.copy()
classes = ["family", "genus", "species"]
features = [col for col in df.columns if col not in classes]

In [ ]:
for c in classes:
    min, max = df[c].min(), df[c].max()
    exp = set(range(min, max + 1))
    present = set(df[c].unique())
    missing = sorted(exp - present)
    unique = len(df[c].unique())
    print("_____________________________________________________________________")
    print(c)
    print(f"Uniques {unique}, Minimum: {min}, Maximum: {max}, Missing: {missing}") 

In [ ]:
for feature in features:
    dracula = len(df[feature].unique())
    if dracula <= 2:
        print(f"{feature}: {dracula}")
    else:
        None

### Base Data Inspection: Key-Takeaways

The labels, whilst being numerical, are just labels, so the line plots might give an understanding of frequency per class pair offers limited insight into the data being presented.

DataFrame Dimensions: 293830 x 536 (R x C)

Constant Columns: None / All columns had at least 2 Unique values

* Class Histograms:
    - Families:
        - 58 Unique
        - 7 familily groups (groups being ~ 3 familiys) saw (less) than 5_000 observations. 
        - 5 familily groups (groups being ~ 3 familiys) saw (more) than 25_000 observations. 
        - Missing families [1, 28]
    - Genus:
        - 191 Unique
        - 8 Genus groups (groups being ~ 9 Geni) saw (less) than 10_000 observations. 
        - 4 Genus groups (groups being ~ 9 Geni) saw (more) than 20_000 observations. 
        - Missing Genus [30, 62, 70, 110, 141]
    - Species:
        - 925 Unique
        - 9 Species groups (groups being ~ 44 Species) saw (less) than 10_000 observations. 
        - 6 Species groups (groups being ~ 44 Species) saw (more) than 20_000 observations.
        - Missing species [114, 229] 

Classes & features are both very skewed and  hierarchal (family -> genus -> species) and multi-class (classes are non-binary) in nature

In [ ]:
def distribution_vs_expected(df: pl.DataFrame, classes: str):
    exp_even = len(df) / df.select(pl.col(classes).n_unique()).item()
    LIMITS = {"family": 60, "genus": 128, "species": 927}
    full   = pl.DataFrame({classes: range(1, LIMITS[classes] + 1)})
    counts = df.group_by(classes).agg(pl.len().alias("n"))
    joined = full.join(counts, on=classes, how="left")
    missing = joined.filter(pl.col("n").is_null()).get_column(classes).to_list()
    dev = (
        joined.with_columns((pl.col("n").fill_null(0) - exp_even).alias("dev"))
            .sort(classes)
    )
    plt.figure(figsize=(11,4))
    plt.bar(dev.get_column(classes).to_list(), dev["dev"].to_list(), width=0.9, label="Actual − Expected")
    plt.axhline(0, linewidth=1)
    handle = Line2D([0],[0], color='none')
    label  = f"Missing {classes}: " + (", ".join(map(str, missing)) if missing else "None")
    plt.legend([handle], [label], loc='center left', bbox_to_anchor=(1.0, 0.5), frameon=False)
    N = LIMITS[classes]
    step = 5 if N <= 60 else (10 if N <= 150 else 50)
    plt.xticks(range(step, N + 1, step))

    plt.xlabel(classes.capitalize()); plt.ylabel("Actual − Expected")
    plt.title(f"Deviation per {classes.capitalize()} (expected ≈ {exp_even:.2f}/{classes})")
    plt.tight_layout()
    plt.show()

In [ ]:
for c in classes:
    data = pl.read_parquet(DATA)
    distribution_vs_expected(data, c)

In [ ]:
import gc 
gc.collect()

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from pathlib import Path

In [ ]:
spec = data.iloc[:, 2].to_numpy()
u, inv = np.unique(spec, return_inverse=True)
cnt = np.bincount(inv)
x1 = np.log1p(cnt)
x2 = np.argsort(np.argsort(cnt)) / (len(cnt) - 1 + 1e-9)
X_all = np.c_[x1, x2]

k = 2
TESTSIZE = 0.4

km = KMeans(n_clusters=k, n_init=10, random_state=0).fit(x1.reshape(-1,1))
order = np.argsort(km.cluster_centers_.ravel())
y_all = np.empty_like(km.labels_)
for r, c in enumerate(order): y_all[km.labels_ == c] = r
X, ins, y, outs = train_test_split(
    X_all, y_all, test_size=TESTSIZE, stratify=y_all, random_state=0
)

scaler = StandardScaler()
X = scaler.fit_transform(X)
ins = scaler.transform(ins)
y, outs = y, outs

KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X, y)
xpred   = KNN.predict(X)
outpred = KNN.predict(ins)
print("Train:", accuracy_score(y, xpred))
print("Test :", accuracy_score(outs, outpred))

In [ ]:
conf = confusion_matrix(outs, outpred)
conf

In [ ]:
x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = KNN.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=FIGSIZE)
plt.contourf(xx, yy, Z, alpha=0.25)
plt.scatter(X[:,0], X[:,1], c=y, s=10, edgecolors='k')
plt.scatter(ins[:,0], ins[:,1], c=outs, s=10, edgecolors='r')
plt.title("KNN rarity (0=rarest)")
plt.xlabel("log1p(count)") 
plt.ylabel("species frequency")

# Run Here TRAIN

In [ ]:
import gc 
gc.collect()

## Train Models

This notebook runs full training pipeline in three stages:

- **Stage 1**: SVM rarity gate (GMM/KMeans/KNN frequency split) with ROC/TPR‑FPR plots and threshold sweep.
- **Stage 2**: Random‑Forest/Subag/Bagging base models per level (Family & Genus) + a Logistic stacker with meta‑features (entropy, margin, optional gate prob). Includes confusion matrix, ROC and PR plots.
- **Stage 3**: Final boosted classifier (AdaBoost, optionally XGBoost if present) on meta‑features from Stage 2 (family/genus stacker probabilities + p_rare).

> Adjust the `DATA` path in libraries tab


In [ ]:
# Libraries
import os
os.environ["OMP_NUM_THREADS"] = "1"; os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"; os.environ["NUMEXPR_NUM_THREADS"] = "1"

import json, time, logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from joblib import dump, load
from collections import Counter

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn.calibration import CalibratedClassifierCV

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

In [ ]:
# Removing Print Overhead
def _setup_logging():
    level = os.getenv("LOGLEVEL", "INFO").upper()
    fmt   = "[%(asctime)s] %(levelname)s - %(message)s"
    date  = "%H:%M:%S"
    logging.basicConfig(level=level, format=fmt, datefmt=date)
    return logging.getLogger("pipeline")

log = _setup_logging()

In [ ]:
RS = 1234
DATA   = Path("Data/master.parquet")
MODELS = Path("Models"); MODELS.mkdir(exist_ok=True)
PLOTS  = Path("Plots");  PLOTS.mkdir(exist_ok=True)
PLOTS_SVM = PLOTS / "SVM"; PLOTS_SVM.mkdir(parents=True, exist_ok=True)
PLOTS_S2  = PLOTS / "Stage2"; PLOTS_S2.mkdir(parents=True, exist_ok=True)
PLOTS_S3  = PLOTS / "Stage3"; PLOTS_S3.mkdir(parents=True, exist_ok=True)

# what kernels to train (names must match factory keys below)
KERNEL_NAMES = [
    "SVMLIN",
    "SVMRBF_APPROX",
    "SVMPoly2"
]

In [ ]:
def load_xy(drop_cols=("family", "genus", "species")):
    t0 = time.perf_counter()
    df = pd.read_parquet(DATA)
    X = df.drop(columns=list(drop_cols), errors="ignore").to_numpy(dtype=np.float32)
    log.info(f"Loaded data: {DATA.name} | X={X.shape} | cols dropped={drop_cols}")
    log.info(f"load_xy took {(time.perf_counter()-t0):.2f}s")
    return df, X

In [ ]:
def s1_initial_rare_labels(species, method="kmeans", random_state=RS):
    t0 = time.perf_counter()
    cnt = Counter(species)
    counts = np.fromiter((cnt[s] for s in species), dtype=np.int32, count=len(species))
    x = np.log1p(counts).reshape(-1, 1)

    if method == "gmm":
        model = GaussianMixture(n_components=2, max_iter=60, random_state=random_state).fit(x)
        rare_comp = np.argmin(model.means_.ravel())
        hard = (model.predict(x) == rare_comp).astype(np.int32)
        soft = model.predict_proba(x)[:, rare_comp]
    elif method == "kmeans":
        model = KMeans(n_clusters=2, n_init=1, random_state=random_state).fit(x)
        rare_comp = np.argmin(model.cluster_centers_.ravel())
        hard = (model.labels_ == rare_comp).astype(np.int32)
        d = np.linalg.norm(x - model.cluster_centers_[rare_comp], axis=1)
        soft = (d.max() - d) / (d.max() - d.min() + 1e-9)
    elif method == "knn":
        k = 10
        model = KNeighborsRegressor(n_neighbors=k).fit(x, counts)
        local_density = model.predict(x)
        soft = 1 - (local_density - local_density.min()) / (local_density.max() - local_density.min() + 1e-9)
        hard = (soft > np.median(soft)).astype(np.int32)
    else:
        raise ValueError("method must be 'gmm','kmeans','knn'")

    log.info(f"initial_rare_labels[{method}] -> hard={hard.mean():.3f} rare_ratio | soft∈[{soft.min():.3f},{soft.max():.3f}] | {(time.perf_counter()-t0):.2f}s")
    return hard, soft

## Stage 1 — SVM Rarity Gate

In [ ]:
# ============================================================
#                            STAGE 1 
# ============================================================

In [ ]:
# --- s1_helpers: SVM 
def s1__gamma_from_dim(n_features):  
    return 1.0 / max(1, n_features)

def s1_SVMLIN(C=1.0):
    base = Pipeline([
        ("mm",  MinMaxScaler()),
        ("lsvc", LinearSVC(C=C, class_weight="balanced", random_state=RS, dual="auto"))
    ])
    base.name = "SVMLIN"
    return base

def s1_SVMRBF_APPROX(n_components=512, gamma=1.0, C=1.0):
    base = Pipeline([
        ("mm",  MinMaxScaler()),
        ("rbf", RBFSampler(gamma=gamma, n_components=n_components, random_state=RS)),
        ("lsvc", LinearSVC(C=C, class_weight="balanced", random_state=RS, dual="auto"))
    ])
    base.name = "SVMRBF_APPROX"
    return base

def s1_SVMPoly_APPROX(degree=2, n_components=512, gamma=1.0, coef0=1.0, C=1.0):
    base = Pipeline([
        ("mm",  MinMaxScaler()),
        ("poly", Nystroem(kernel="polynomial", degree=degree, gamma=gamma, coef0=coef0,
                          n_components=n_components, random_state=RS)),
        ("lsvc", LinearSVC(C=C, class_weight="balanced", random_state=RS, dual="auto"))
    ])
    base.name = f"SVMPoly{degree}"
    return base

def s1_build_svm(name, n_features, cfg=None):
    cfg = cfg or {}
    gamma = cfg.get("gamma", s1__gamma_from_dim(n_features))
    C     = cfg.get("C", 1.0)
    ncomp = cfg.get("n_components", 512)
    if name == "SVMLIN":
        return s1_SVMLIN(C=C)
    if name == "SVMRBF_APPROX":
        return s1_SVMRBF_APPROX(n_components=ncomp, gamma=gamma, C=C)
    if name.startswith("SVMPoly"):
        d = int(name.replace("SVMPoly",""))
        return s1_SVMPoly_APPROX(degree=d, n_components=ncomp, gamma=gamma, coef0=cfg.get("coef0",1.0), C=C)
    raise RuntimeError(f"unknown svm name: {name}")

def s1__svm_tags(methods, kernels, train_size):
    pct = int(train_size*100)
    return [f"{m}_{k}_{pct}" for m in methods for k in kernels]

def s1_ready(methods, kernels, train_size):
    tags = s1__svm_tags(methods, kernels, train_size)
    ok_all = all((MODELS/"SVM"/f"{t}.joblib").exists() and (MODELS/"SVM"/f"{t}_meta.json").exists()
                 for t in tags)
    best_ok = (MODELS/"SVM"/"_best.json").exists()
    return ok_all and best_ok

def s1_rebuild_best_from_metas(methods, kernels, train_size):
    metas = list((MODELS/"SVM").glob("*_meta.json"))
    if not metas: return None
    best = None
    allowed_prefixes = [f"{m}_{k}_{int(train_size*100)}" for m in methods for k in kernels]
    for m in metas:
        tag = m.stem.replace("_meta","")
        if tag not in allowed_prefixes:
            continue
        info = json.load(open(m, "r", encoding="utf-8"))
        if (best is None) or (info.get("auc",0.0) > best["auc"]):
            parts = tag.split("_")
            best = {"tag": tag, "method": parts[0], "name": "_".join(parts[1:-1]),
                    **info}
    if best:
        json.dump(best, open(MODELS/"SVM"/"_best.json","w",encoding="utf-8"), indent=2)
    return best

# --- s1_plots
def s1_plot_cm_binary(y_true, y_pred, out_path, title):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    fig, ax = plt.subplots(figsize=(5,4))
    im = ax.imshow(cm, cmap="Blues")
    for (i,j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha="center", va="center", fontsize=10)
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(["Common(0)","Rare(1)"]); ax.set_yticklabels(["Common(0)","Rare(1)"])
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_title(title); fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout()
    fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)

def s1_plot_tpr_fpr(thr, tpr, fpr, out_path, title):
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(thr, tpr, label="TPR", color="red", lw=1.6)
    ax.plot(thr, fpr, label="FPR", color="blue", lw=1.6)
    ax.plot(thr, (tpr - fpr) - 0.5, label="TPR−FPR (offset −0.5)", color="k", lw=1.6)
    ax.invert_xaxis(); ax.set_xlabel("Threshold"); ax.set_ylabel("Rate / Offset Rate")
    ax.set_title(title); ax.legend(frameon=False); ax.grid(alpha=0.3)
    fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)
    
def s1_plot_counts_vs_threshold(y_true, proba, out_png, title, step=0.001, out_csv=None):
    """
    Plots TP, TN, FP, FN counts vs threshold in [0,1].
    step=0.001 -> 1001 points. Use 1e-4 if you want ultra-fine (slower).
    """
    y_true = np.asarray(y_true, dtype=np.int32)
    proba  = np.asarray(proba,  dtype=np.float32)

    n_steps = int(round(1.0/step)) + 1
    thr = np.linspace(0.0, 1.0, n_steps)

    TP = np.empty_like(thr, dtype=np.int64)
    TN = np.empty_like(thr, dtype=np.int64)
    FP = np.empty_like(thr, dtype=np.int64)
    FN = np.empty_like(thr, dtype=np.int64)

    pos = (y_true == 1)
    neg = ~pos

    # straight, clear, fast-enough loop
    for i, t in enumerate(thr):
        pred = (proba >= t)
        TP[i] = np.sum(pred & pos)
        TN[i] = np.sum(~pred & neg)
        FP[i] = np.sum(pred & neg)
        FN[i] = np.sum(~pred & pos)

    # plot
    fig, ax = plt.subplots(figsize=(9, 5))
    ax.plot(thr, TP, label="TP")
    ax.plot(thr, TN, label="TN")
    ax.plot(thr, FP, label="FP")
    ax.plot(thr, FN, label="FN")
    ax.set_xlabel("Threshold")
    ax.set_ylabel("Count")
    ax.set_title(title)
    ax.legend(frameon=False)
    ax.grid(alpha=0.3)
    fig.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)

    if out_csv:
        pd.DataFrame({"threshold": thr, "TP": TP, "TN": TN, "FP": FP, "FN": FN}).to_csv(out_csv, index=False)

In [ ]:
# --- s1_train
def s1_train(train_size=0.75, methods=("gmm","kmeans","knn"),
             kernels=KERNEL_NAMES, calib_cv=2, max_train=None,
             ncomp_rbf=512, ncomp_poly=512, C=1.0):
    log.info(f"[Stage1] Start | train_size={train_size} methods={list(methods)} kernels={list(kernels)}")
    t0 = time.perf_counter()

    df, X = load_xy()
    n_features = X.shape[1]
    species = df["species"].to_numpy()
    model_dir = MODELS / "SVM"; model_dir.mkdir(parents=True, exist_ok=True)

    best = None
    for method in methods:
        y_rare, _ = s1_initial_rare_labels(species, method=method)
        Xtr, Xte, ytr, yte = train_test_split(X, y_rare, test_size=(1-train_size), stratify=y_rare, random_state=RS)
        log.info(f"[Stage1] Method={method} | split: train={Xtr.shape[0]} test={Xte.shape[0]}")

        if max_train is not None and Xtr.shape[0] > max_train:
            rng = np.random.default_rng(RS)
            idx = rng.choice(Xtr.shape[0], size=max_train, replace=False)
            Xtr = Xtr[idx]; ytr = ytr[idx]
            log.info(f"[Stage1] Subsampled train -> {Xtr.shape[0]} rows")

        for name in kernels:
            cfg = {"C": C, "gamma": s1__gamma_from_dim(n_features)}
            if name == "SVMRBF_APPROX":
                cfg["n_components"] = ncomp_rbf
            if name.startswith("SVMPoly"):
                cfg["n_components"] = ncomp_poly
            base = s1_build_svm(name, n_features, cfg=cfg)
            clf  = CalibratedClassifierCV(estimator=base, method="sigmoid", cv=calib_cv)

            tag  = f"{method}_{name}_{int(train_size*100)}"
            job  = model_dir / f"{tag}.joblib"
            meta = model_dir / f"{tag}_meta.json"

            t1 = time.perf_counter()
            clf.fit(Xtr, ytr)
            proba = clf.predict_proba(Xte)[:, 1]
            fpr, tpr, thr = roc_curve(yte, proba)
            auc = roc_auc_score(yte, proba)
            t_opt = thr[np.argmax(tpr - fpr)]
            yhat = (proba >= t_opt).astype(np.int32)

            dump(clf, job)
            with open(meta, "w", encoding="utf-8") as f:
                json.dump({
                    "method": method, "name": name, "auc": float(auc),
                    "threshold": float(t_opt),
                    "gamma": float(cfg["gamma"]), "C": float(C),
                    "n_components": int(cfg.get("n_components", 0)),
                    "degree": int(name.replace("SVMPoly","")) if name.startswith("SVMPoly") else None
                }, f, indent=2)

            s1_plot_tpr_fpr(thr, tpr, fpr, PLOTS_SVM / f"{tag}_tprfpr.png", f"{tag} TPR/FPR")
            s1_plot_cm_binary(yte, yhat, PLOTS_SVM / f"{tag}_cm.png", title=f"{tag} Confusion")
            s1_plot_counts_vs_threshold(
                y_true=yte,
                proba=proba,
                out_png=PLOTS_SVM / f"{tag}_tp_tn_fp_fn.png",
                title=f"{tag} TP/TN/FP/FN vs Threshold",
                step=0.001,  # set to 1e-4 if you want finer; will be slower
                out_csv=PLOTS_SVM / f"{tag}_tp_tn_fp_fn.csv"
            )

            log.info(f"[Stage1] {tag} | AUC={auc:.3f} thr={t_opt:.3f} | fit+eval {(time.perf_counter()-t1):.2f}s")
            if (best is None) or (auc > best["auc"]):
                best = {"tag": tag, "method": method, "name": name,
                        "auc": float(auc), "threshold": float(t_opt),
                        "gamma": float(cfg["gamma"]), "C": float(C),
                        "n_components": int(cfg.get("n_components", 0)),
                        "degree": int(name.replace("SVMPoly","")) if name.startswith("SVMPoly") else None}

    json.dump(best, open(model_dir / "_best.json","w",encoding="utf-8"), indent=2)
    log.info(f"[Stage1] Best={best['tag']} AUC={best['auc']:.3f} thr={best['threshold']:.3f} | total {(time.perf_counter()-t0):.2f}s")
    return best

In [ ]:
# --- s1_make_oof_prare - OUT OF FOLD PREDICTION / CROSS VAL
def s1_make_oof_prare(best, n_splits=3):
    out_path = MODELS / "SVM" / "p_rare_oof.npy"
    if out_path.exists():
        log.info(f"[Stage1-OOF] Skip: found {out_path.name}")
        return np.load(out_path)

    log.info(f"[Stage1-OOF] Start | best={best['tag']} | folds={n_splits}")
    df, X = load_xy()
    species = df["species"].to_numpy()
    y_rare, _ = s1_initial_rare_labels(species, method=best["method"])

    cfg = {
        "gamma": float(best.get("gamma", s1__gamma_from_dim(X.shape[1]))),
        "C": float(best.get("C", 1.0)),
        "n_components": int(best.get("n_components", 512)),
    }
    base = s1_build_svm(best["name"], X.shape[1], cfg=cfg)
    clf  = CalibratedClassifierCV(estimator=base, method="sigmoid", cv=2)

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RS)
    p_rare_oof = cross_val_predict(clf, X, y_rare, cv=cv, method="predict_proba", n_jobs=1)[:, 1]
    np.save(out_path, p_rare_oof)
    log.info(f"[Stage1-OOF] Saved OOF p_rare -> {out_path}")
    return p_rare_oof

## Stage 2 — Family / Genus Ensembles

In [ ]:
# ============================================================
#                            STAGE 2 
# ============================================================

In [ ]:
def s2_ready():
    return (MODELS/"Stage2_family_stacker.joblib").exists() and \
           (MODELS/"Stage2_genus_stacker.joblib").exists() and \
           (MODELS/"Stage2_family_oof.npy").exists() and \
           (MODELS/"Stage2_genus_oof.npy").exists()

def s2__fix_prob(a: np.ndarray) -> np.ndarray:
    a = np.asarray(a, dtype=np.float64)
    a = np.nan_to_num(a, nan=0.0, posinf=0.0, neginf=0.0)
    s = a.sum(axis=1, keepdims=True)
    good = s.squeeze() > 0
    a[good] = a[good] / s[good]
    return a

def _fit_rf_with_fallback(label: str, X, y, base_params: dict) -> RandomForestClassifier:
    tries = [
        {},  # try base_params first
        {"n_jobs": 1, "max_samples": 0.6, "max_depth": 18, "min_samples_leaf": 6},
        {"n_jobs": 1, "max_samples": 0.5, "max_depth": 16, "min_samples_leaf": 8,
         "n_estimators": max(100, int(base_params["n_estimators"] * 0.75))},
        {"n_jobs": 1, "max_samples": 0.4, "max_depth": 14, "min_samples_leaf": 10,
         "n_estimators": max(80, int(base_params["n_estimators"] * 0.5))}
    ]

    last_err = None
    for i, tweak in enumerate(tries, 1):
        params = {**base_params, **tweak}
        try:
            rf = RandomForestClassifier(**params)
            t0 = time.perf_counter()
            rf.fit(X, y)
            dt = time.perf_counter() - t0
            oob = getattr(rf, "oob_score_", float("nan"))
            log.info(
                "[Stage2] %s RF fit try#%d %.2fs | classes=%d | oob_acc=%.4f | params=%s",
                label, i, dt, len(rf.classes_), oob,
                {
                    "n_estimators": params.get("n_estimators"),
                    "n_jobs": params.get("n_jobs", -1),
                    "max_samples": params.get("max_samples"),
                    "max_depth": params.get("max_depth"),
                    "min_samples_leaf": params.get("min_samples_leaf", 1),
                },
            )
            return rf
        except MemoryError as e:
            last_err = e
            log.warning(f"[Stage2] {label} RF try#{i} ran out of memory; tightening settings and retrying…")
            import gc; gc.collect()
    raise last_err

In [ ]:
def s2_train_oob(n_estimators_fam=300, n_estimators_gen=250):
    if s2_ready():
        log.info("[Stage2] Skip: all artefacts found")
        return (np.load(MODELS/"Stage2_family_oof.npy"), np.load(MODELS/"Stage2_genus_oof.npy"))

    log.info(f"[Stage2] Start | n_estimators_fam={n_estimators_fam} n_estimators_gen={n_estimators_gen}")
    df, X = load_xy()
    yF = df["family"].to_numpy()
    yG = df["genus"].to_numpy()

    fam_base = dict(
        n_estimators=n_estimators_fam,
        bootstrap=True, oob_score=True,
        max_features="sqrt",
        max_depth=18,               
        min_samples_leaf=4,          
        max_samples=0.6,          
        n_jobs=2,                  
        random_state=RS
    )
    gen_base = dict(
        n_estimators=n_estimators_gen,
        bootstrap=True, oob_score=True,
        max_features="sqrt",
        max_depth=16,              
        min_samples_leaf=6,
        max_samples=0.5,
        n_jobs=2,
        random_state=RS
    )
    fam = _fit_rf_with_fallback("Family", X, yF, fam_base)
    import gc; gc.collect()
    gen = _fit_rf_with_fallback("Genus",  X, yG, gen_base)

    fam_oob = s2__fix_prob(fam.oob_decision_function_)
    gen_oob = s2__fix_prob(gen.oob_decision_function_)

    dump(fam, MODELS / "Stage2_family_stacker.joblib")
    dump(gen, MODELS / "Stage2_genus_stacker.joblib")
    np.save(MODELS / "Stage2_family_oof.npy", fam_oob)
    np.save(MODELS / "Stage2_genus_oof.npy", gen_oob)

    log.info("[Stage2] Saved models + OOB probs")
    return fam_oob, gen_oob

In [ ]:
def s2__map_labels_to_index(y, classes):
    idx = {c:i for i,c in enumerate(classes)}
    return np.array([idx[v] for v in y], dtype=np.int32)

def s2__plot_confusion_top(name, y_true, y_pred, classes, outdir, topN=20):
    cnt = Counter(y_true); top = [c for c,_ in cnt.most_common(topN)]
    sel = np.isin(y_true, top)
    idx = {c:i for i,c in enumerate(classes)}
    lab = [idx[c] for c in top]
    cm = confusion_matrix([idx[v] for v in y_true[sel]], [idx[v] for v in y_pred[sel]], labels=lab)
    fig, ax = plt.subplots(figsize=(0.45*len(lab)+4, 0.45*len(lab)+4))
    im = ax.imshow(cm, aspect="auto")
    ax.set_xticks(range(len(lab))); ax.set_yticks(range(len(lab)))
    ax.set_xticklabels([str(classes[i]) for i in lab], rotation=90)
    ax.set_yticklabels([str(classes[i]) for i in lab])
    ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title(f"{name} — Confusion (top {topN})")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.savefig(outdir / f"{name}_confusion_top{topN}.png", dpi=300, bbox_inches="tight"); plt.close(fig)

# ---------- helpers (no seaborn, simple, fast) ----------
def _y_idx(y_true, classes):
    m = {c:i for i,c in enumerate(classes)}
    return np.array([m[v] for v in y_true], dtype=np.int32)

def s2_topk_accuracy(y_true, proba, classes, ks=(1,3,5)):
    y = _y_idx(y_true, classes)
    order = np.argsort(proba, axis=1)[:, ::-1]
    out = {}
    for k in ks:
        hit = (order[:, :k] == y[:, None]).any(axis=1)
        out[k] = float(hit.mean())
    return out

def s2_reliability_ece(y_true, proba, classes, bins=20, out_path=None, title="Reliability"):
    y = _y_idx(y_true, classes)
    conf = proba.max(axis=1)
    pred = proba.argmax(axis=1)
    correct = (pred == y).astype(np.float64)

    edges = np.linspace(0, 1, bins+1)
    idx = np.clip(np.digitize(conf, edges) - 1, 0, bins-1)
    bin_acc  = np.zeros(bins); bin_conf = np.zeros(bins); bin_cnt = np.zeros(bins)

    for b in range(bins):
        sel = (idx == b)
        n = sel.sum()
        if n:
            bin_cnt[b]  = n
            bin_acc[b]  = correct[sel].mean()
            bin_conf[b] = conf[sel].mean()

    N = len(conf)
    ece = float(np.sum((bin_cnt / N) * np.abs(bin_acc - bin_conf)))

    if out_path is not None:
        fig, ax = plt.subplots(figsize=(6,5))
        ax.plot([0,1],[0,1], ls="--", lw=1, label="Perfect")
        ax.plot(bin_conf, bin_acc, marker="o", lw=1.5, label=f"Model (ECE={ece:.3f})")
        ax.set_xlabel("Confidence"); ax.set_ylabel("Empirical accuracy")
        ax.set_title(title); ax.legend(frameon=False); ax.grid(alpha=0.3)
        fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)
    return ece

def s2_confidence_hist(y_true, proba, classes, out_path, title="Confidence histogram"):
    y = _y_idx(y_true, classes)
    conf = proba.max(axis=1)
    pred = proba.argmax(axis=1)
    correct = (pred == y)
    fig, ax = plt.subplots(figsize=(7,4))
    ax.hist(conf[correct], bins=30, alpha=0.6, label="Correct")
    ax.hist(conf[~correct], bins=30, alpha=0.6, label="Incorrect")
    ax.set_xlabel("Max predicted probability"); ax.set_ylabel("Count")
    ax.set_title(title); ax.legend(frameon=False); ax.grid(alpha=0.2)
    fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)

def s2_coverage_accuracy(y_true, proba, classes, out_path, title="Coverage vs Accuracy"):
    y = _y_idx(y_true, classes)
    conf = proba.max(axis=1)
    pred = proba.argmax(axis=1)
    correct = (pred == y)
    thr = np.linspace(0, 1, 101)
    cov = []; acc = []
    for t in thr:
        keep = conf >= t
        cov.append(keep.mean())
        acc.append(correct[keep].mean() if keep.any() else np.nan)
    cov = np.array(cov); acc = np.array(acc)

    fig, ax1 = plt.subplots(figsize=(7,4))
    ax1.plot(thr, cov, lw=1.5, label="Coverage")
    ax1.set_xlabel("Threshold on max prob"); ax1.set_ylabel("Coverage")
    ax1.grid(alpha=0.2)
    ax2 = ax1.twinx()
    ax2.plot(thr, acc, lw=1.5, ls="--", label="Accuracy")
    ax2.set_ylabel("Accuracy")
    fig.suptitle(title)
    fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)

def s2_class_acc_vs_support(y_true, y_pred, out_path, title="Class accuracy vs support"):
    cnt = Counter(y_true)
    per_class = sorted(cnt.keys(), key=lambda c: cnt[c], reverse=True)
    acc = []
    sup = []
    for c in per_class:
        sel = (y_true == c)
        acc.append((y_pred[sel] == c).mean())
        sup.append(cnt[c])
    acc = np.array(acc); sup = np.array(sup)

    fig, ax = plt.subplots(figsize=(7,4))
    ax.scatter(np.log1p(sup), acc, s=12, alpha=0.7)
    ax.set_xlabel("log(1 + support)"); ax.set_ylabel("Per-class accuracy")
    ax.set_title(title); ax.grid(alpha=0.2)
    fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)

def s2_dump_top_confusions(y_true, y_pred, out_csv, top=40):
    from collections import defaultdict
    mis = defaultdict(int)
    for t, p in zip(y_true, y_pred):
        if t != p: mis[(t,p)] += 1
    rows = sorted(mis.items(), key=lambda kv: kv[1], reverse=True)[:top]
    import csv
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["true","pred","count"])
        for (t,p), c in rows: w.writerow([t,p,c])

def s2_plot_feature_importance(model, feat_names, out_path, topn=30, title="Feature importance (top)"):
    imp = getattr(model, "feature_importances_", None)
    if imp is None: return
    idx = np.argsort(imp)[::-1][:topn]
    names = [feat_names[i] for i in idx]
    vals  = imp[idx]
    fig, ax = plt.subplots(figsize=(8, 0.28*len(names)+1))
    ax.barh(range(len(names)), vals[::-1])
    ax.set_yticks(range(len(names))); ax.set_yticklabels(names[::-1])
    ax.invert_yaxis()
    ax.set_xlabel("Importance"); ax.set_title(title)
    fig.tight_layout()
    fig.savefig(out_path, dpi=300, bbox_inches="tight"); plt.close(fig)

# ---------- main Stage-2 evaluation ----------
def s2_eval_visuals():
    df, _ = load_xy()
    yF = df["family"].to_numpy()
    yG = df["genus"].to_numpy()

    fam = load(MODELS / "Stage2_family_stacker.joblib")
    gen = load(MODELS / "Stage2_genus_stacker.joblib")
    fam_oob = np.load(MODELS / "Stage2_family_oof.npy")   # normalized earlier
    gen_oob = np.load(MODELS / "Stage2_genus_oof.npy")

    # Predictions (OOB)
    fam_pred = fam.classes_[fam_oob.argmax(axis=1)]
    gen_pred = gen.classes_[gen_oob.argmax(axis=1)]

    # Keep your confusion figures
    s2__plot_confusion_top("Family", yF, fam_pred, fam.classes_, PLOTS_S2, topN=40)
    s2__plot_confusion_top("Genus",  yG, gen_pred, gen.classes_, PLOTS_S2, topN=80)

    # Feature names for importances
    feat_names = df.drop(columns=["family","genus","species"], errors="ignore").columns.tolist()

    # ---- Family metrics/plots ----
    tkF  = s2_topk_accuracy(yF, fam_oob, fam.classes_, ks=(1,3,5))
    eceF = s2_reliability_ece(yF, fam_oob, fam.classes_, bins=20,
                              out_path=PLOTS_S2 / "Family_reliability.png",
                              title="Family — Reliability")
    s2_confidence_hist(yF, fam_oob, fam.classes_, PLOTS_S2 / "Family_conf_hist.png",
                       title="Family — Confidence (correct vs incorrect)")
    s2_coverage_accuracy(yF, fam_oob, fam.classes_, PLOTS_S2 / "Family_cov_acc.png",
                         title="Family — Coverage vs Accuracy")
    s2_class_acc_vs_support(yF, fam_pred, PLOTS_S2 / "Family_acc_vs_support.png",
                            title="Family — Per-class accuracy vs support")
    s2_dump_top_confusions(yF, fam_pred, PLOTS_S2 / "Family_top_confusions.csv", top=40)
    s2_plot_feature_importance(fam, feat_names, PLOTS_S2 / "Family_feature_importance.png",
                               topn=30, title="Family — Feature importance")

    log.info("[Stage2/Family] TopK=%s | ECE=%.4f", tkF, eceF)

    # ---- Genus metrics/plots ----
    tkG  = s2_topk_accuracy(yG, gen_oob, gen.classes_, ks=(1,3,5))
    eceG = s2_reliability_ece(yG, gen_oob, gen.classes_, bins=20,
                              out_path=PLOTS_S2 / "Genus_reliability.png",
                              title="Genus — Reliability")
    s2_confidence_hist(yG, gen_oob, gen.classes_, PLOTS_S2 / "Genus_conf_hist.png",
                       title="Genus — Confidence (correct vs incorrect)")
    s2_coverage_accuracy(yG, gen_oob, gen.classes_, PLOTS_S2 / "Genus_cov_acc.png",
                         title="Genus — Coverage vs Accuracy")
    s2_class_acc_vs_support(yG, gen_pred, PLOTS_S2 / "Genus_acc_vs_support.png",
                            title="Genus — Per-class accuracy vs support")
    s2_dump_top_confusions(yG, gen_pred, PLOTS_S2 / "Genus_top_confusions.csv", top=40)
    s2_plot_feature_importance(gen, feat_names, PLOTS_S2 / "Genus_feature_importance.png",
                               topn=30, title="Genus — Feature importance")

    log.info("[Stage2/Genus] TopK=%s | ECE=%.4f", tkG, eceG)
    log.info("[Stage2] Visuals saved → Plots/Stage2")

In [ ]:
# ============================================================
#                            STAGE 3 
# ============================================================

In [ ]:
def s3_ready():
    return (MODELS/"Stage3_Booster.joblib").exists() and (MODELS/"Stage3_AdaBoost.joblib").exists()

def s3__build_metaX_y():
    df, _ = load_xy()
    yS = df["species"].to_numpy()

    p_rare = np.load(MODELS / "SVM" / "p_rare_oof.npy")
    fam_oob = np.load(MODELS / "Stage2_family_oof.npy")
    gen_oob = np.load(MODELS / "Stage2_genus_oof.npy")

    n = min(len(p_rare), len(fam_oob), len(gen_oob), len(yS))
    metaX = np.hstack([p_rare[:n].reshape(-1, 1), fam_oob[:n], gen_oob[:n]])
    y = yS[:n]
    return metaX, y

def s3_plot_confusion_top(y_true, y_pred, out_dir, topN=30):
    classes = np.unique(np.concatenate([y_true, y_pred]))
    cnt = Counter(y_true); top = [c for c,_ in cnt.most_common(topN)]
    sel = np.isin(y_true, top)
    idx = {c:i for i,c in enumerate(classes)}
    lab = [idx[c] for c in top]
    cm = confusion_matrix([idx[v] for v in y_true[sel]], [idx[v] for v in y_pred[sel]], labels=lab)

    fig, ax = plt.subplots(figsize=(0.45*len(lab)+4, 0.45*len(lab)+4))
    im = ax.imshow(cm, aspect="auto")
    ax.set_xticks(range(len(lab))); ax.set_yticks(range(len(lab)))
    ax.set_xticklabels([str(classes[i]) for i in lab], rotation=90)
    ax.set_yticklabels([str(classes[i]) for i in lab])
    ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title("Species — Confusion (top classes)")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.savefig(out_dir / "Species_confusion_top.png", dpi=300, bbox_inches="tight"); plt.close(fig)
    log.info("[Stage3] Visual saved → %s", out_dir / "Species_confusion_top.png")
    
def s3_train():
    path = MODELS / "Stage3_Booster.joblib"
    if path.exists():
        log.info("[Stage3] Skip: booster already exists")
        return load(path)

    log.info("[Stage3] Start (HistGB)")
    metaX, y = s3__build_metaX_y()
    log.info(f"[Stage3] metaX={metaX.shape} | y={y.shape}")

    Xtr, Xte, ytr, yte = train_test_split(metaX, y, test_size=0.2, stratify=y, random_state=RS)

    booster = HistGradientBoostingClassifier(learning_rate=0.1, max_iter=100, early_stopping=True, random_state=RS)
    t1 = time.perf_counter(); booster.fit(Xtr, ytr); log.info(f"[Stage3] Booster fit {(time.perf_counter()-t1):.2f}s")

    yhat = booster.predict(Xte)
    acc  = accuracy_score(yte, yhat)
    log.info(f"[Stage3] HistGB holdout acc={acc:.4f}")
    dump(booster, path)
    log.info("[Stage3] Saved %s", path)

    s3_plot_confusion_top(yte, yhat, out_dir=PLOTS_S3, topN=30)
    return booster

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

def s3_plot_adaboost_curves(model, X_val, y_val, out_dir):
    from sklearn.metrics import log_loss

    acc1, acc5, ll = [], [], []
    classes = model.classes_
    cls_to_idx = {c:i for i,c in enumerate(classes)}
    y_idx = np.fromiter((cls_to_idx[v] for v in y_val), dtype=np.int32, count=len(y_val))

    for proba in model.staged_predict_proba(X_val):
        pred_top1 = np.argmax(proba, axis=1)
        acc1.append((pred_top1 == y_idx).mean())

        k = min(5, proba.shape[1])
        topk_idx = np.argpartition(proba, -k, axis=1)[:, -k:]
        correct_topk = np.any(topk_idx == y_idx[:, None], axis=1)
        acc5.append(correct_topk.mean())

        # use consistent label order for log_loss
        ll.append(log_loss(y_val, proba, labels=classes))

    it = np.arange(1, len(acc1) + 1)

    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(it, acc1, label="Top-1 Acc")
    ax.plot(it, acc5, label="Top-5 Acc")
    ax.set_xlabel("Boosting Iteration"); ax.set_ylabel("Accuracy"); ax.set_title("AdaBoost — Accuracy vs Iteration")
    ax.legend(frameon=False); ax.grid(alpha=0.3)
    fig.savefig(out_dir / "AdaBoost_acc_vs_iter.png", dpi=300, bbox_inches="tight"); plt.close(fig)

    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(it, ll, label="LogLoss")
    ax.set_xlabel("Boosting Iteration"); ax.set_ylabel("LogLoss"); ax.set_title("AdaBoost — LogLoss vs Iteration")
    ax.legend(frameon=False); ax.grid(alpha=0.3)
    fig.savefig(out_dir / "AdaBoost_logloss_vs_iter.png", dpi=300, bbox_inches="tight"); plt.close(fig)

    pd.DataFrame({"iter": it, "acc_top1": acc1, "acc_top5": acc5, "logloss": ll}) \
      .to_csv(out_dir / "AdaBoost_iter_metrics.csv", index=False)

    log.info("[Stage3] Saved AdaBoost iteration curves → %s", out_dir)


In [ ]:
def s3_train_adaboost(n_estimators=400, learning_rate=0.5):
    path = (MODELS / "Stage3_AdaBoost.joblib")
    if path.exists():
        log.info("[Stage3] Skip: AdaBoost already exists")
        return load(path)

    log.info("[Stage3] Start (AdaBoost)")
    metaX, y = s3__build_metaX_y()
    Xtr, Xte, ytr, yte = train_test_split(metaX, y, test_size=0.2, stratify=y, random_state=RS)

    base = DecisionTreeClassifier(max_depth=1, random_state=RS)
    ada = AdaBoostClassifier(
        estimator=base,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=RS
    )
    t0 = time.perf_counter(); ada.fit(Xtr, ytr)
    log.info("[Stage3] AdaBoost fit %.2fs", time.perf_counter() - t0)

    yhat = ada.predict(Xte)
    acc = accuracy_score(yte, yhat)
    log.info("[Stage3] AdaBoost holdout acc=%.4f", acc)

    dump(ada, path)
    log.info("[Stage3] Saved %s", path)

    s3_plot_confusion_top(yte, yhat, out_dir=PLOTS_S3, topN=30)
    s3_plot_adaboost_curves(ada, Xte, yte, PLOTS_S3)
    return ada

## Col Mapping: For testing on Headerless Frame


In [ ]:
TRAIN_PATH = Path("Data/master.parquet")   
MODELS     = Path("Models"); MODELS.mkdir(parents=True, exist_ok=True)
COLMAP_OUT = MODELS / "colmap.json"
DROP_COLS  = ("family", "genus", "species")

In [ ]:
def _read_any(p: Path) -> pd.DataFrame:
    if p.suffix.lower() == ".parquet":
        return pd.read_parquet(p)
    # default CSV with header row 0; edit if needed
    return pd.read_csv(p, sep=",", header=0, low_memory=False)

def save_colmap():
    if not TRAIN_PATH.exists():
        raise SystemExit(f"missing training file: {TRAIN_PATH}")
    df = _read_any(TRAIN_PATH)
    feat = [c for c in df.columns if c not in DROP_COLS]
    COLMAP_OUT.write_text(
        json.dumps({"features": feat, "drop": list(DROP_COLS), "count": len(feat)}, indent=2),
        encoding="utf-8"
    )
    print(f"[schema] wrote {COLMAP_OUT} | features={len(feat)}")

In [ ]:
save_colmap()

# Calling Full Training Iteration

In [ ]:
t_all = time.perf_counter()

TRAIN_SIZE = 0.75
METHODS = ("gmm", "kmeans", "knn")
KERNELS = KERNEL_NAMES  # ["SVMLIN","SVMRBF_APPROX","SVMPoly2"]
(PLOTS_S3).mkdir(parents=True, exist_ok=True)


# -------- Stage 1 --------
if s1_ready(METHODS, KERNELS, TRAIN_SIZE):
    log.info("[Stage1] Skip: all SVM models present")
    best = json.load(open(MODELS / "SVM" / "_best.json", "r", encoding="utf-8"))
else:
    best = s1_rebuild_best_from_metas(METHODS, KERNELS, TRAIN_SIZE)
    if best is None:
        best = s1_train(
            train_size=TRAIN_SIZE,
            methods=METHODS,
            kernels=KERNELS,
            calib_cv=2,
            max_train=None,
            ncomp_rbf=512,
            ncomp_poly=512,
            C=1.0
        )
    else:
        log.info(f"[Stage1] Rebuilt best from metas -> {best['tag']} (AUC={best['auc']:.3f})")

# S1 OOF p_rare (used later by Stage 3)
_ = s1_make_oof_prare(best, n_splits=3)

# -------- Stage 2 --------
fam_oob, gen_oob = s2_train_oob(n_estimators_fam=300, n_estimators_gen=250)

s2_eval_visuals()

# -------- Stage 3 --------
_ = s3_train()                           
# _ = s3_train_adaboost(400, 0.5) - ADABOOST Alterantive        

log.info(f"[Stage3 ALL] {(time.perf_counter()-t0):.2f}s")
    
log.info(f"[ALL] Total runtime {(time.perf_counter() - t_all):.2f}s")

KeyboardInterrupt: 

# STOPPER FOR TRAINING 

# Test

In [ ]:
import os, json, time
from pathlib import Path
import numpy as np
import pandas as pd
from joblib import load

Set:
* INPUT_PATH
* OUT_CSV Path

In [ ]:
# ---- CONFIG ----
# Input Path can Handle both Parquet and CSV files but expects a full merged dataset (with removed classes)
# Refer to squish() in the preliminary data analysis.

INPUT_PATH  = Path("Data/master.parquet")   

# Squish method utilises headers, ensure no headers in the new dataset
HEADERLESS  = False                 

# If CSV
CSV_SEP     = ","                   
OUT_CSV     = Path("predictions.csv")
PRINT_HEAD  = 5

MODELS = Path("Models")
SVM_DIR = MODELS / "SVM"
COLMAP_PATH = MODELS / "colmap.json"

In [ ]:
# Test Helpers
def _must(p: Path):
    if not p.exists():
        raise SystemExit(f"missing: {p}")
    return p

def _read_any(p: Path, headerless: bool, sep: str) -> pd.DataFrame:
    if p.suffix.lower() == ".parquet":
        return pd.read_parquet(p)
    header = None if headerless else 0
    return pd.read_csv(p, sep=sep, header=header, low_memory=False)

def _to_float32(a: pd.DataFrame) -> np.ndarray:
    return a.apply(pd.to_numeric, errors="coerce").fillna(0.0).to_numpy(np.float32, copy=False)

def _align_X(df: pd.DataFrame, feat: list[str], headered: bool, drop_if_present: list[str]) -> np.ndarray:
    if headered:
        keep = [c for c in df.columns if c not in drop_if_present]
        df = df[keep]
        return _to_float32(df.reindex(columns=feat, fill_value=0.0))
    if df.shape[1] != len(feat):
        raise SystemExit(f"headerless cols={df.shape[1]} but schema expects {len(feat)}")
    return _to_float32(df)

In [ ]:
def _load_artifacts():
    _must(COLMAP_PATH)
    cm = json.loads(COLMAP_PATH.read_text(encoding="utf-8"))
    feat = cm["features"]
    drop = cm.get("drop", ["family","genus","species"])

    best_meta = json.loads(_must(SVM_DIR / "_best.json").read_text(encoding="utf-8"))
    svm_gate  = load(_must(SVM_DIR / f"{best_meta['tag']}.joblib"))

    fam = load(_must(MODELS / "Stage2_family_stacker.joblib"))
    gen = load(_must(MODELS / "Stage2_genus_stacker.joblib"))
    booster = load(_must(MODELS / "Stage3_Booster.joblib"))
    return feat, drop, svm_gate, fam, gen, booster

In [ ]:
def predict():
    feat, drop, svm_gate, fam, gen, booster = _load_artifacts()
    df_new = _read_any(_must(INPUT_PATH), headerless=HEADERLESS, sep=CSV_SEP)
    X = _align_X(df_new, feat, headered=(not HEADERLESS), drop_if_present=drop)

    t0 = time.perf_counter()

    # Stage 1 — rarity gate
    if not hasattr(svm_gate, "predict_proba"):
        raise SystemExit("SVM gate must be calibrated with predict_proba")
    p_rare = svm_gate.predict_proba(X)[:, 1]

    # Stage 2 — family/genus
    fam_proba = fam.predict_proba(X); fam_pred = fam.classes_[fam_proba.argmax(axis=1)]; fam_conf = fam_proba.max(axis=1)
    gen_proba = gen.predict_proba(X); gen_pred = gen.classes_[gen_proba.argmax(axis=1)]; gen_conf = gen_proba.max(axis=1)

    # Stage 3 — species
    metaX = np.hstack([p_rare.reshape(-1,1), fam_proba, gen_proba])
    species_pred = booster.predict(metaX)
    species_conf = booster.predict_proba(metaX).max(axis=1) if hasattr(booster, "predict_proba") else np.ones(len(species_pred), dtype=np.float32)

    out = pd.DataFrame({
        "p_rare": p_rare,
        "family_pred": fam_pred,  "family_conf": fam_conf,
        "genus_pred":  gen_pred,  "genus_conf":  gen_conf,
        "species_pred": species_pred, "species_conf": species_conf,
    })
    out.to_csv(OUT_CSV, index=False)
    dt = time.perf_counter() - t0
    print(f"[predict] rows={len(out)} | wrote {OUT_CSV} | {dt:.2f}s")
    if PRINT_HEAD:
        print(out.head(PRINT_HEAD).to_string(index=False))

In [ ]:
predict()